# Repartitioning DataFrames

Partitions are a central concept in Apache Spark. They are used for distributing and parallelizing work onto different executors, which run on multiple servers. 

### Determining Partitions
Basically Spark uses two different strategies for splitting up data into multiple partitions:
1. When Spark loads data, the records are put into partitions along natural borders. For example every HDFS block (and thereby every file) is represented by a different partition. Therefore the number of partitions of a DataFrame read from disk is solely determined by the number of HDFS blocks
2. Certain operations like `JOIN`s and aggregations require that records with the same key are physically in the same partition. This is achieved by a shuffle phase. The number of partitions is specified by the global Spark configuration variable `spark.sql.shuffle.partitions` which has a default value of 200.

### Repartitiong Data
Since partitions have a huge influence on the execution, Spark also allows you to explicitly change the partitioning schema of a DataFrame. This makes sense only in a very limited (but still important) set of cases, which we will discuss in this notebook.

### Weather Example
Surprise, surprise, we will again use the weather example and see what explicit repartitioning gives us.

### Disable Automatic Broadcast JOINs
In order to see the shuffle operations, we need to prevent Spark from executiong `JOIN` operations as broadcast joins. Again this can be turned off by setting the Spark configuration variable `spark.sql.autoBroadcastJoinThreshold` to -1.

In [8]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

# 1 Load Data

First we load the weather data, which consists of the measurement data and some station metadata.

In [1]:
storageLocation = "s3://dimajix-training/data/weather"

## 1.1 Load Measurements

Measurements are stored in multiple directories (one per year). But we will limit ourselves to a single year in the analysis to improve readability of execution plans.

In [2]:
from pyspark.sql.functions import *
from functools import reduce

# Read in all years, store them in an Python array
raw_weather_per_year = [spark.read.text(storageLocation + "/" + str(i)).withColumn("year", lit(i)) for i in range(2003,2015)]

# Union all years together
raw_weather = reduce(lambda l,r: l.union(r), raw_weather_per_year)                        

Use a single year to keep execution plans small

In [3]:
raw_weather = spark.read.text(storageLocation + "/2003").withColumn("year", lit(2003))

### Extract Measurements

Measurements were stored in a proprietary text based format, with some values at fixed positions. We need to extract these values with a simple `SELECT` statement.

In [4]:
weather = raw_weather.select(
    col("year"),
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban"),
    substring(col("value"),16,8).alias("date"),
    substring(col("value"),24,4).alias("time"),
    substring(col("value"),42,5).alias("report_type"),
    substring(col("value"),61,3).alias("wind_direction"),
    substring(col("value"),64,1).alias("wind_direction_qual"),
    substring(col("value"),65,1).alias("wind_observation"),
    (substring(col("value"),66,4).cast("float") / lit(10.0)).alias("wind_speed"),
    substring(col("value"),70,1).alias("wind_speed_qual"),
    (substring(col("value"),88,5).cast("float") / lit(10.0)).alias("air_temperature"),
    substring(col("value"),93,1).alias("air_temperature_qual")
)

## 1.2 Load Station Metadata

We also need to load the weather station meta data containing information about the geo location, country etc of individual weather stations.

In [5]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

# 2 Partitions

Since partitions is a concept at the RDD level and a DataFrame per se does not contain an RDD, we need to access the RDD in order to inspect the number of partitions.

In [6]:
weather.rdd.getNumPartitions()

5

## 2.1 Repartitioning Data

You can repartition any DataFrame by specifying the target number of partitions and the partitioning columns. While it should be clear what *number of partitions* actually means, the term *partitionng columns* might require some explanation.

### Partitioning Columns
Except for the case when Spark initially reads data, all DataFrames are partitioned along *partitioning columns*, which means that all records having the same values in the corresponding columns will end up in the same partition. Spark implicitly performs such repartitioning as shuffle operations for `JOIN`s and grouped aggregation (except when a DataFrame already has the correct partitioning columns and number of partitions)

### Manual Repartitioning
As already mentioned, you can explicitly repartition a DataFrame using teh `repartition()` method.

In [7]:
weather_rep = weather.repartition(10, weather["usaf"], weather["wban"])
weather_rep.rdd.getNumPartitions()

10

## 2.2 Repartition & Joins

As already mentioned, Spark implicitly performs a repartitioning aka shuffle for `JOIN` operations.

### Execution Plan

So let us inspect the execution plan of a `JOIN` operation.

In [9]:
result = weather.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result.explain()

== Physical Plan ==
*(5) SortMergeJoin [usaf#87, wban#88], [usaf#122, wban#123], Inner
:- *(2) Sort [usaf#87 ASC NULLS FIRST, wban#88 ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(usaf#87, wban#88, 200)
:     +- *(1) Project [2003 AS year#84, substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#88, substring(value#82, 16, 8) AS date#89, substring(value#82, 24, 4) AS time#90, substring(value#82, 42, 5) AS report_type#91, substring(value#82, 61, 3) AS wind_direction#92, substring(value#82, 64, 1) AS wind_direction_qual#93, substring(value#82, 65, 1) AS wind_observation#94, (cast(cast(substring(value#82, 66, 4) as float) as double) / 10.0) AS wind_speed#95, substring(value#82, 70, 1) AS wind_speed_qual#96, (cast(cast(substring(value#82, 88, 5) as float) as double) / 10.0) AS air_temperature#97, substring(value#82, 93, 1) AS air_temperature_qual#98]
:        +- *(1) Filter (isnotnull(substring(value#82, 11, 5)) && isnotnull(substring(value#82, 5, 6)))
:    

### Remarks

As we already discussed, each `JOIN` is executed with the following steps
1. Filter `NULL` values (it's an inner join)
2. Repartition DataFrame on the join columns with 200 partitions
3. Sort each partition independently
4. Perform a `SortMergeJoin`

### Pre-partition data (first try)

Now let us try if we can cache the shuffle (repartition) and sort operation. This is useful in cases, where you have to perform multiple joins on the same set of columns, for example with different DataFrames.

So let's simply repartition the `weather` DataFrame on the two columns `usaf` and `wban`. We also have to use 200 partitions, because this is what Spark will use for `JOIN` operations.

In [9]:
weather_rep = weather.repartition(200, weather["usaf"], weather["wban"])
weather_rep.cache()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string]

#### Execution Plan

Let's analyze the resulting execution plan. Ideally all the preparation work before the `SortMergeJoin` happens before the `cache` operation.

In [10]:
result = weather_rep.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result.explain()

== Physical Plan ==
*(4) SortMergeJoin [usaf#87, wban#88], [usaf#122, wban#123], Inner
:- *(1) Sort [usaf#87 ASC NULLS FIRST, wban#88 ASC NULLS FIRST], false, 0
:  +- *(1) Filter (isnotnull(wban#88) && isnotnull(usaf#87))
:     +- InMemoryTableScan [year#84, usaf#87, wban#88, date#89, time#90, report_type#91, wind_direction#92, wind_direction_qual#93, wind_observation#94, wind_speed#95, wind_speed_qual#96, air_temperature#97, air_temperature_qual#98], [isnotnull(wban#88), isnotnull(usaf#87)]
:           +- InMemoryRelation [year#84, usaf#87, wban#88, date#89, time#90, report_type#91, wind_direction#92, wind_direction_qual#93, wind_observation#94, wind_speed#95, wind_speed_qual#96, air_temperature#97, air_temperature_qual#98], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
:                 +- Exchange hashpartitioning(usaf#87, wban#88, 200)
:                    +- *(1) Project [2003 AS year#84, substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#

#### Remarks
We did not reach completely what we wanted. The `sort` and `filter` operation still occur after the cache.

### Pre-partition data (second try)

We already partially achieved our goal of caching all preparational work of the `SortMergeJoin`, but the sorting was still preformed after the caching. So let's try to insert an appropriate sort operation.

In [19]:
# Release cache to simplify execution plan
weather.unpersist()

weather_rep = weather.repartition(200, weather["usaf"], weather["wban"]) \
    .orderBy(weather["usaf"], weather["wban"])
weather_rep.cache()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string]

#### Execution Plan

In [20]:
result = weather_rep.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result.explain()

== Physical Plan ==
*(5) SortMergeJoin [usaf#87, wban#88], [usaf#122, wban#123], Inner
:- *(2) Sort [usaf#87 ASC NULLS FIRST, wban#88 ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(usaf#87, wban#88, 200)
:     +- *(1) Filter (isnotnull(wban#88) && isnotnull(usaf#87))
:        +- InMemoryTableScan [year#84, usaf#87, wban#88, date#89, time#90, report_type#91, wind_direction#92, wind_direction_qual#93, wind_observation#94, wind_speed#95, wind_speed_qual#96, air_temperature#97, air_temperature_qual#98], [isnotnull(wban#88), isnotnull(usaf#87)]
:              +- InMemoryRelation [year#84, usaf#87, wban#88, date#89, time#90, report_type#91, wind_direction#92, wind_direction_qual#93, wind_observation#94, wind_speed#95, wind_speed_qual#96, air_temperature#97, air_temperature_qual#98], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
:                    +- *(2) Sort [usaf#87 ASC NULLS FIRST, wban#88 ASC NULLS FIRST], true, 0
:                       +- Exchange rang

#### Remarks

We actually created a worse situation: Now we have two sort operations! Definately not what we wanted to have.

So let's think for a moment: The `SortMergeJoin` requires that each partition is sorted, but after the repartioning occured. The `orderBy` operation we used above will create a global order over all partitions (and thereby destroy all the repartition work immediately). So we need something else, which still keeps the current partitions but only sort in each partition independently.

### Pre-partition data (final try)

Fortunately Spark provides a `sortWithinPartitions` method, which does exactly what it sounds like.

In [21]:
# Release cache to simplify execution plan
weather.unpersist()

weather_rep = weather.repartition(200, weather["usaf"], weather["wban"]) \
    .sortWithinPartitions(weather["usaf"], weather["wban"])
weather_rep.cache()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string]

#### Execution Plan

In [23]:
result = weather_rep.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result.explain()

== Physical Plan ==
*(4) SortMergeJoin [usaf#87, wban#88], [usaf#122, wban#123], Inner
:- *(1) Filter (isnotnull(wban#88) && isnotnull(usaf#87))
:  +- InMemoryTableScan [year#84, usaf#87, wban#88, date#89, time#90, report_type#91, wind_direction#92, wind_direction_qual#93, wind_observation#94, wind_speed#95, wind_speed_qual#96, air_temperature#97, air_temperature_qual#98], [isnotnull(wban#88), isnotnull(usaf#87)]
:        +- InMemoryRelation [year#84, usaf#87, wban#88, date#89, time#90, report_type#91, wind_direction#92, wind_direction_qual#93, wind_observation#94, wind_speed#95, wind_speed_qual#96, air_temperature#97, air_temperature_qual#98], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
:              +- *(2) Sort [usaf#87 ASC NULLS FIRST, wban#88 ASC NULLS FIRST], false, 0
:                 +- Exchange hashpartitioning(usaf#87, wban#88, 200)
:                    +- *(1) Project [2003 AS year#84, substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS

#### Remarks

That looks really good. The filter operation is still executed after the cache, but that cannot be cached such that Spark uses this information.

So whenever you want to prepartition data, you need to execute the following steps:
* repartition with the join columns and default number of partitions
* sortWithinPartitions with the join columns
* probably cache (otherwise there is no benefit at all)

### Inspect WebUI

We can also inspect the WebUI and see how everything is executed.

Phase 1: Build cache

In [23]:
result.count()

1798753

Phase 2: Use cache

In [24]:
result.count()

1798753

# 3 Repartition & Aggregations

Similar to `JOIN` operations, Spark also requires an appropriate partitioning in grouped aggregations. Again, we can use the same strategy and appropriateky prepartition data in cases where multiple joins and aggregations are performed using the same columns.

## 3.1 Simple Aggregation

So let's perform the usual aggregation (but this time without a previous `JOIN`) with groups defined by the station id (`usaf` and `wban`).

In [25]:
result = weather.groupBy(weather["usaf"], weather["wban"]).agg(
        min(when(weather.air_temperature_qual == lit(1), weather.air_temperature)).alias('min_temp'),
        max(when(weather.air_temperature_qual == lit(1), weather.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[usaf#87, wban#88], functions=[min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
+- Exchange hashpartitioning(usaf#87, wban#88, 200)
   +- *(1) HashAggregate(keys=[usaf#87, wban#88], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), partial_max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
      +- *(1) Project [substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#88, (cast(cast(substring(value#82, 88, 5) as float) as double) / 10.0) AS air_temperature#97, substring(value#82, 93, 1) AS air_temperature_qual#98]
         +- *(1) FileScan text [value#82] Batched: false, Format: Text, Location: InMemoryFileIndex[s3://dimajix-training/data/weather/2003], PartitionFilters: [], PushedFilters: [], ReadSchema: struct

### Remarks
Each grouped aggregation is executed with the following steps:
1. Perform partial aggregation (`HashAggregate`)
2. Shuffle intermediate result (`Exchange hashpartitioning`)
3. Perform final aggregation (`HashAggregate`)

## 3.2 Aggregation after repartition

Now let us perform the same aggregation, but this time let's use the preaggregated weather data set `weather_rep` instead.

In [26]:
result = weather_rep.groupBy(weather["usaf"], weather["wban"]).agg(
        min(when(weather_rep.air_temperature_qual == lit(1), weather_rep.air_temperature)).alias('min_temp'),
        max(when(weather_rep.air_temperature_qual == lit(1), weather_rep.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(1) HashAggregate(keys=[usaf#87, wban#88], functions=[min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
+- *(1) HashAggregate(keys=[usaf#87, wban#88], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), partial_max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
   +- InMemoryTableScan [usaf#87, wban#88, air_temperature#97, air_temperature_qual#98]
         +- InMemoryRelation [year#84, usaf#87, wban#88, date#89, time#90, report_type#91, wind_direction#92, wind_direction_qual#93, wind_observation#94, wind_speed#95, wind_speed_qual#96, air_temperature#97, air_temperature_qual#98], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
               +- *(2) Sort [usaf#87 ASC NULLS FIRST, wban#88 ASC NULLS FIRST], false, 0
                  +- E

### Remarks
Spark obviously detects the correct partitioning of the cached `weather_rep` DataFrame. The sorting actually is not required, but does not hurt either (except performance...). Therefore only two steps are executed after the cache operation:
1. Partial aggregation (`HashAggregate`)
2. Final aggregation (`HashAggregate`)

But note that although you saved a shuffle operation of partial aggregates, in most cases it is not adviseable to prepartition data only for aggregations for the following reasons:
* You could perform all aggregations in a single `groupBy` and `agg` chain
* In most cases the preaggregated data is significantly smaller than the original data, therefore the shuffle doesn't hurt that much

# 4 Interaction between Join, Aggregate & Repartition

Now we have seen two operations which require a shuffle of the data. Of course Spark is clever enough to avoid an additional shuffle operation in chains of `JOIN` and grouped aggregations, which use the same aggregation columns.

## 4.1 Aggregation after Join on same key

So let's see what happens with a grouped aggregation after a join operation.

In [27]:
joined = weather.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result = joined.groupBy(weather["usaf"], weather["wban"]).agg(
        min(when(joined.air_temperature_qual == lit(1), joined.air_temperature)).alias('min_temp'),
        max(when(joined.air_temperature_qual == lit(1), joined.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(5) HashAggregate(keys=[usaf#87, wban#88], functions=[min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
+- *(5) HashAggregate(keys=[usaf#87, wban#88], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), partial_max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
   +- *(5) Project [usaf#87, wban#88, air_temperature#97, air_temperature_qual#98]
      +- *(5) SortMergeJoin [usaf#87, wban#88], [usaf#122, wban#123], Inner
         :- *(2) Sort [usaf#87 ASC NULLS FIRST, wban#88 ASC NULLS FIRST], false, 0
         :  +- Exchange hashpartitioning(usaf#87, wban#88, 200)
         :     +- *(1) Project [substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#88, (cast(cast(substring(value#82, 88, 5) as float) as double) / 10.0) AS air_temperature#

### Remarks

As you can see, Spark performs a single shuffle operation. The order of operation is as follows:
1. Filter `NULL` values (it's an inner join)
2. Shuffle data on `usaf` and `wban`
3. Sort partitions by `usaf` and `wban`
4. Perform `SortMergeJoin`
5. Perform partial aggregation `HashAggregate`
6. Perform final aggregation `HashAggregate`

## 4.2 Aggregation after Join using repartitioned data

Of course we can also use the pre-repartitioned weather DataFrame. This will work as expected, Spark does not add any additional shuffle operation.

In [28]:
joined = weather_rep.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result = joined.groupBy(weather["usaf"], weather["wban"]).agg(
        min(when(joined.air_temperature_qual == lit(1), joined.air_temperature)).alias('min_temp'),
        max(when(joined.air_temperature_qual == lit(1), joined.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(4) HashAggregate(keys=[usaf#87, wban#88], functions=[min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
+- *(4) HashAggregate(keys=[usaf#87, wban#88], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), partial_max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
   +- *(4) Project [usaf#87, wban#88, air_temperature#97, air_temperature_qual#98]
      +- *(4) SortMergeJoin [usaf#87, wban#88], [usaf#122, wban#123], Inner
         :- *(1) Filter (isnotnull(wban#88) && isnotnull(usaf#87))
         :  +- InMemoryTableScan [usaf#87, wban#88, air_temperature#97, air_temperature_qual#98], [isnotnull(wban#88), isnotnull(usaf#87)]
         :        +- InMemoryRelation [year#84, usaf#87, wban#88, date#89, time#90, report_type#91, wind_direction#92, wind_direction_qu

## 4.3 Aggregation after Join with different key

So far we only looked at join and grouping operations using the same keys. If we use different keys (for example the country) in both operations, we expect Spark to add an additional shuffle operations. Let's see...

In [29]:
joined = weather.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result = joined.groupBy(stations["ctry"]).agg(
        min(when(joined.air_temperature_qual == lit(1), joined.air_temperature)).alias('min_temp'),
        max(when(joined.air_temperature_qual == lit(1), joined.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(6) HashAggregate(keys=[ctry#125], functions=[min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
+- Exchange hashpartitioning(ctry#125, 200)
   +- *(5) HashAggregate(keys=[ctry#125], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), partial_max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
      +- *(5) Project [air_temperature#97, air_temperature_qual#98, CTRY#125]
         +- *(5) SortMergeJoin [usaf#87, wban#88], [usaf#122, wban#123], Inner
            :- *(2) Sort [usaf#87 ASC NULLS FIRST, wban#88 ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(usaf#87, wban#88, 200)
            :     +- *(1) Project [substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#88, (cast(cast(substring(value#82, 88, 5) as float) 

## 4.4 Aggregation after Broadcast-Join 

If we use a broadcast join instead of a sort merge join, the we will have a shuffle operation for the aggregation again (since the broadcast join just avoids the shuffle). Let's verify that theory...

In [30]:
joined = weather.join(broadcast(stations), (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result = joined.groupBy(weather["usaf"], weather["wban"]).agg(
        min(when(joined.air_temperature_qual == lit(1), joined.air_temperature)).alias('min_temp'),
        max(when(joined.air_temperature_qual == lit(1), joined.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(3) HashAggregate(keys=[usaf#87, wban#88], functions=[min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
+- Exchange hashpartitioning(usaf#87, wban#88, 200)
   +- *(2) HashAggregate(keys=[usaf#87, wban#88], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END), partial_max(CASE WHEN (cast(air_temperature_qual#98 as int) = 1) THEN air_temperature#97 END)])
      +- *(2) Project [usaf#87, wban#88, air_temperature#97, air_temperature_qual#98]
         +- *(2) BroadcastHashJoin [usaf#87, wban#88], [usaf#122, wban#123], Inner, BuildRight
            :- *(2) Project [substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#88, (cast(cast(substring(value#82, 88, 5) as float) as double) / 10.0) AS air_temperature#97, substring(value#82, 93, 1) AS air_temperature_qual#98]
            : 

# 5 Coalesce

There is another use case for changing the number of partitions: Writing results to HDFS/S3/whatever. Per design Spark writes each partition into a separate file, and there is no way around that. But when partitions do not contain many records, this may not only be ugly, but also unperformant and might cause additional trouble. Specifically currently HDFS is not designed to handle many small files, but prefers fewer large files instead.

Therefore it is often desireable to reduce the number of partitions of a DataFrame just before writing the result to disk. You could perform this task by a `repartition` operation, but this is an expensive operation requiring an additional shuffle operation. Therefore Spark provides an additional method called `coalesce` which can be used to reduce the number of partitions without incurring an additional shuffle. Spark simply logically concatenates multiple partitions into new partitions.

### Inspect Number of Partitions

For this example, we will use the `weather_rep` DataFrame, which contains exactly 200 partitions.

In [38]:
weather_rep.rdd.getNumPartitions()

200

## 5.1 Merge Partitions using coalesce

In order to reduce the number of partitions, we simply use the `coalesce` method.

In [31]:
weather_small = weather_rep.coalesce(16)
weather_small.explain()

== Physical Plan ==
Coalesce 16
+- InMemoryTableScan [year#84, usaf#87, wban#88, date#89, time#90, report_type#91, wind_direction#92, wind_direction_qual#93, wind_observation#94, wind_speed#95, wind_speed_qual#96, air_temperature#97, air_temperature_qual#98]
      +- InMemoryRelation [year#84, usaf#87, wban#88, date#89, time#90, report_type#91, wind_direction#92, wind_direction_qual#93, wind_observation#94, wind_speed#95, wind_speed_qual#96, air_temperature#97, air_temperature_qual#98], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
            +- *(2) Sort [usaf#87 ASC NULLS FIRST, wban#88 ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(usaf#87, wban#88, 200)
                  +- *(1) Project [2003 AS year#84, substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#88, substring(value#82, 16, 8) AS date#89, substring(value#82, 24, 4) AS time#90, substring(value#82, 42, 5) AS report_type#91, substring(value#82, 61, 3) AS wind_d

### Inspect WebUI

In [40]:
weather_rep.count()

1798753

## 5.2 Saving files

We already discussed that Spark writes a separate file per partition. So let's see the result when we write the `weather_rep` DataFrame containing 200 partitions.

### Write 200 Partitions

In [41]:
weather_rep.write.mode("overwrite").parquet("/tmp/weather_rep")

#### Inspect the Result
Using a simple HDFS CLI util, we can inspect the result on HDFS.

In [43]:
%%bash
hdfs dfs -ls /tmp/weather_rep

Found 91 items
-rw-r--r--   1 hadoop hadoop          0 2018-10-07 07:17 /tmp/weather_rep/_SUCCESS
-rw-r--r--   1 hadoop hadoop       1337 2018-10-07 07:16 /tmp/weather_rep/part-00000-19014412-a1e6-4348-a41d-49986590b40b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop      24241 2018-10-07 07:16 /tmp/weather_rep/part-00003-19014412-a1e6-4348-a41d-49986590b40b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop      63340 2018-10-07 07:17 /tmp/weather_rep/part-00005-19014412-a1e6-4348-a41d-49986590b40b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop      32695 2018-10-07 07:17 /tmp/weather_rep/part-00006-19014412-a1e6-4348-a41d-49986590b40b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop     126661 2018-10-07 07:17 /tmp/weather_rep/part-00011-19014412-a1e6-4348-a41d-49986590b40b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop      89610 2018-10-07 07:17 /tmp/weather_rep/part-00013-19014412-a1e6-4348-a41d-49986590b40b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop      73063 2018-10-07

### Write 16 Partitions

Now let's write the `coalesce`d DataFrame and inspect the result on HDFS

In [42]:
weather_small.write.mode("overwrite").parquet("/tmp/weather_small")

#### Inspect Result

In [44]:
%%bash
hdfs dfs -ls /tmp/weather_small

Found 17 items
-rw-r--r--   1 hadoop hadoop          0 2018-10-07 07:17 /tmp/weather_small/_SUCCESS
-rw-r--r--   1 hadoop hadoop     290888 2018-10-07 07:17 /tmp/weather_small/part-00000-31d2cdf1-532c-4549-b706-d040d0a0921b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop     539188 2018-10-07 07:17 /tmp/weather_small/part-00001-31d2cdf1-532c-4549-b706-d040d0a0921b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop     490533 2018-10-07 07:17 /tmp/weather_small/part-00002-31d2cdf1-532c-4549-b706-d040d0a0921b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop     338415 2018-10-07 07:17 /tmp/weather_small/part-00003-31d2cdf1-532c-4549-b706-d040d0a0921b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop     460959 2018-10-07 07:17 /tmp/weather_small/part-00004-31d2cdf1-532c-4549-b706-d040d0a0921b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop     358779 2018-10-07 07:17 /tmp/weather_small/part-00005-31d2cdf1-532c-4549-b706-d040d0a0921b-c000.snappy.parquet
-rw-r--r--   1 hadoop hadoop     394